## Imports

In [1]:
import numpy as np 

## Reading Databases

In [2]:
class Vertex():
    def __init__(self, name):
        self.name = name
        self.edges = []
        self.weights = []
        self.pheromone = [] 
    
    def add(self, other, weight):
        self.edges.append(other)
        self.weights.append(weight)
        self.pheromone.append(10)  # Inicialize pheromone equals 10 to all edges

In [3]:
class Graph():
    def __init__(self):
        self.vertices = dict()
    
    def build_graph(self, file):
        with open(file) as f:
            lines = f.readlines()
            lines = [line.strip() for line in lines]
            for line in lines:
                line = line.split('\t')
                line = list(map(float, line))
                if line[0] not in self.vertices.keys():
                    self.vertices[line[0]] = Vertex(line[0])
                self.vertices[line[0]].add(line[1], line[2])
    
    def update_pheromone(self, origin, destine, cost):
            index = self.vertices[origin].edges.index(destine)
            self.vertices[origin].pheromone[index] += cost
    
    def evaporate_pheromone(self, evaporation_rate):
        for vertex in self.vertices.keys():
            #self.vertices[vertex].pheromone -= np.array(evaporation_rate)
            self.vertices[vertex].pheromone *= np.array(1-evaporation_rate)
            self.vertices[vertex].pheromone = np.clip(self.vertices[vertex].pheromone, a_min=0, a_max=None)
    
    def show_pheromone(self):
        for vertex in self.vertices.keys():
            print(self.vertices[vertex].pheromone)
    
    def reset_pheromone(self):
        for vertex in self.vertices.keys():
            self.vertices[vertex].pheromone = [10 for _ in self.vertices[vertex].pheromone]

In [4]:
file = 'graph1.txt'
G1 = Graph()
G1.build_graph(file)

## ACO 

In [5]:
class Ant():
    def __init__(self, destine):
        self.path = [1]
        self.destine = destine
    
    def show(self):
        for e in self.path[:-1]:
            print(e, end=', ')
        print(self.path[-1])
    
    def step(self, graph, alfa=1, beta=2):
        vertex = self.path[-1]
        edges = graph.vertices[vertex].edges
        pheromone = graph.vertices[vertex].pheromone
        weights = graph.vertices[vertex].weights
        index = [i for (i,e) in enumerate(edges) if e not in self.path]
        if index == []:
            
            return None, 0
        #edges = [e for (i,e) in enumerate(edges) if i in index]
        pheromone = [p for (i,p) in enumerate(pheromone) if i in index]
        v_weights = [w for (i,w) in enumerate(weights) if i in index]
        #print(edges)
        prob_num = np.array(pheromone)**alfa * np.array(v_weights)**beta
        prob = prob_num/np.sum(prob_num)
        chosen = np.random.choice(index, p=prob)
        destination = edges[chosen]
        cost = weights[chosen]
        self.path.append(destination)
        return destination, cost
        
    def walk(self, graph, alfa=1, beta=2):
        total_cost = 0
        destination = 1
        while(destination != self.destine and destination != None):
            destination, cost = self.step(graph, alfa, beta)
            # print(destination, len(self.path))
            ##Test
            ##if len(self.path) <= 99:
            ##     if destination == 100:
            ##            self.path.pop()
            ##            destination = 0
            ##            continue
            ## 
            total_cost += cost

        #self.show()
        if destination == None:
            print("DEAD END ")
            self.path=[1]  # Clear ant path before return 
            return 0, None
        #print("Total lenght = ", len(self.path))
        #print("Total weight = ", total_cost)
        
        # Reset ant path
        path = self.path 
        self.path = [1]  # Clear ant path before return 
        
        return total_cost, path


In [6]:
class ACO():
    def __init__(self, graph=None, number_of_ants=20, max_iterations=100, evaporation_rate=0.05, alfa=1, beta=2):
        self.number_of_ants = number_of_ants
        self.graph = graph
        self.ants = [Ant(len(self.graph.vertices)) for _ in range(self.number_of_ants)]
        self.max_iterations = max_iterations
        self.evaporation_rate = evaporation_rate
        self.current_iteration = 0
        self.best = (0, None)
        self.alfa = alfa
        self.beta = beta
        
    def iteration(self):
        self.current_iteration += 1
        
        #self.graph.show_pheromone()
        #Evaporate pheromone
        self.graph.evaporate_pheromone(self.evaporation_rate)
        
        fitness = []
        for ant in self.ants:
            
            cost, path = ant.walk(self.graph, self.alfa, self.beta)
            fitness.append(cost)
            if path == None:
                continue
            
            #Update best 
            if cost > self.best[0]:
                self.best = (cost, path)
        
            #Update pheromone
            for i, vertex in enumerate(path[:-1]):
                origin = vertex
                destine = path[i+1]
                self.graph.update_pheromone(origin, destine, cost/100)
        
        #print("Best so far: ", self.best[0])
        print("Best so far: {},  Average fitness: {}".format(self.best[0], np.mean(fitness)))
        
        # Return iteration fitness (mean and best) 
        return np.mean(fitness), self.best[0]
    
    def run(self):
        best_list = []
        mean_list = []
        for i in range(self.max_iterations):
            mean, best = self.iteration()
            mean_list.append(mean)
            best_list.append(best)
        return mean_list, best_list
    
    def reset(self):
        self.current_iteration = 0
        self.best = (0, None)
        self.graph.reset_pheromone()

In [7]:
fitness = [1,3, 4,5, 6]
x = np.mean(fitness)
x

3.8

## Problem 1

In [8]:
sol1 = ACO(G1)

In [9]:
number_of_ants = 20
max_iterations = 150
evaporation_rate = 0.05
sol1.run()

Best so far: 753.0,  Average fitness: 287.3
Best so far: 777.0,  Average fitness: 408.55
DEAD END 
Best so far: 777.0,  Average fitness: 359.65
Best so far: 783.0,  Average fitness: 393.85
Best so far: 783.0,  Average fitness: 357.45
Best so far: 784.0,  Average fitness: 325.4
Best so far: 808.0,  Average fitness: 308.35
Best so far: 808.0,  Average fitness: 383.0
Best so far: 808.0,  Average fitness: 385.45
Best so far: 842.0,  Average fitness: 396.3
Best so far: 842.0,  Average fitness: 341.3
Best so far: 842.0,  Average fitness: 317.25
Best so far: 842.0,  Average fitness: 392.95
Best so far: 842.0,  Average fitness: 409.45
Best so far: 842.0,  Average fitness: 324.3
Best so far: 842.0,  Average fitness: 403.95
Best so far: 842.0,  Average fitness: 419.05
Best so far: 880.0,  Average fitness: 380.15
Best so far: 880.0,  Average fitness: 377.1
Best so far: 880.0,  Average fitness: 440.5
Best so far: 880.0,  Average fitness: 266.75
Best so far: 880.0,  Average fitness: 331.25
Best so 

([287.3,
  408.55,
  359.65,
  393.85,
  357.45,
  325.4,
  308.35,
  383.0,
  385.45,
  396.3,
  341.3,
  317.25,
  392.95,
  409.45,
  324.3,
  403.95,
  419.05,
  380.15,
  377.1,
  440.5,
  266.75,
  331.25,
  383.1,
  331.3,
  383.8,
  330.55,
  322.55,
  346.05,
  315.7,
  288.2,
  323.7,
  268.45,
  377.2,
  347.9,
  292.15,
  421.1,
  380.4,
  374.5,
  301.45,
  392.85,
  301.75,
  283.4,
  314.15,
  268.4,
  363.35,
  273.1,
  304.2,
  295.9,
  329.45,
  265.65,
  342.7,
  291.15,
  387.75,
  312.2,
  290.05,
  257.6,
  342.25,
  311.0,
  302.85,
  333.95,
  313.8,
  333.0,
  337.9,
  360.65,
  295.8,
  330.55,
  289.9,
  262.0,
  314.3,
  365.15,
  368.7,
  325.1,
  351.8,
  404.6,
  335.1,
  327.05,
  343.5,
  342.6,
  372.85,
  336.9,
  317.0,
  317.1,
  304.6,
  295.9,
  290.1,
  434.55,
  366.4,
  289.7,
  293.65,
  299.0,
  339.7,
  320.5,
  376.3,
  327.4,
  415.75,
  320.2,
  430.05,
  355.45,
  366.6,
  305.7],
 [753.0,
  777.0,
  777.0,
  783.0,
  783.0,
  784.0,
  8

## Problem 2

In [10]:
file = 'graph2.txt'
G2 = Graph()
G2.build_graph(file)

In [11]:
number_of_ants = 20
max_iterations = 150
evaporation_rate = 0.01
alfa = 1
beta = 2
sol2 = ACO(G2, number_of_ants, max_iterations, evaporation_rate, alfa, beta)

In [12]:
sol2.reset()
sol2.run()

Best so far: 136.0,  Average fitness: 74.85
Best so far: 136.0,  Average fitness: 69.5
DEAD END 
Best so far: 136.0,  Average fitness: 66.7
DEAD END 
Best so far: 136.0,  Average fitness: 54.2
Best so far: 136.0,  Average fitness: 50.8
DEAD END 
Best so far: 136.0,  Average fitness: 52.45
Best so far: 136.0,  Average fitness: 64.0
Best so far: 136.0,  Average fitness: 68.45
DEAD END 
Best so far: 136.0,  Average fitness: 51.15
Best so far: 136.0,  Average fitness: 64.2
DEAD END 
Best so far: 139.0,  Average fitness: 57.8
Best so far: 139.0,  Average fitness: 65.9
DEAD END 
Best so far: 139.0,  Average fitness: 60.55
Best so far: 139.0,  Average fitness: 70.0
Best so far: 139.0,  Average fitness: 71.65
Best so far: 139.0,  Average fitness: 57.3
Best so far: 139.0,  Average fitness: 61.3
DEAD END 
Best so far: 139.0,  Average fitness: 63.55
Best so far: 139.0,  Average fitness: 72.0
DEAD END 
Best so far: 139.0,  Average fitness: 60.15
Best so far: 139.0,  Average fitness: 67.7
Best so f

([74.85,
  69.5,
  66.7,
  54.2,
  50.8,
  52.45,
  64.0,
  68.45,
  51.15,
  64.2,
  57.8,
  65.9,
  60.55,
  70.0,
  71.65,
  57.3,
  61.3,
  63.55,
  72.0,
  60.15,
  67.7,
  61.95,
  57.8,
  74.3,
  73.9,
  68.5,
  64.8,
  59.95,
  69.85,
  56.2,
  75.75,
  60.05,
  56.25,
  67.25,
  64.85,
  55.5,
  63.85,
  66.25,
  54.85,
  72.05,
  71.55,
  68.85,
  56.1,
  60.5,
  62.05,
  64.9,
  65.35,
  56.9,
  70.3,
  61.45,
  66.15,
  67.5,
  72.3,
  62.1,
  57.0,
  72.55,
  57.9,
  73.4,
  67.3,
  58.15,
  62.1,
  59.9,
  83.25,
  73.45,
  63.55,
  69.65,
  53.65,
  63.6,
  64.55,
  76.05,
  63.95,
  65.25,
  47.1,
  62.8,
  69.9,
  54.9,
  65.45,
  72.45,
  60.8,
  80.95,
  59.35,
  65.4,
  63.8,
  59.55,
  65.4,
  59.45,
  66.4,
  70.0,
  78.25,
  72.85,
  65.65,
  59.25,
  51.0,
  82.4,
  77.15,
  77.4,
  69.8,
  73.4,
  60.6,
  71.65,
  69.6,
  67.95,
  55.35,
  70.45,
  69.0,
  89.0,
  56.55,
  73.9,
  65.2,
  64.8,
  76.75,
  53.25,
  77.35,
  73.6,
  71.8,
  71.35,
  63.6,
  65.35